Yalda Foroutan

https://github.com/Youlenda

**Similarity NN for Hand Poses Classification**

In [ ]:
##installing tensorflow, keras and efficientnet

# !pip install q tensorflow==2.1
# !pip install q keras==2.3.1

# !pip install efficientnet

In [21]:
##importing libraries

import os
import glob 
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import keras
from keras import backend as k 
from keras.backend import sigmoid
from keras import applications
from keras import optimizers
from keras.optimizers import Adam
import efficientnet.keras as enet
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.utils.generic_utils import get_custom_objects
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Activation

Download Dataset.

[Training Set](https://drive.google.com/file/d/1eo7kkq8zzrlWgcCcQh1Stkei_eZhIAbl/view?usp=sharing)


[Validation Set](https://drive.google.com/file/d/1sghCxu83xV_DIZ1qWA4eLg_-hAQh-Pxs/view?usp=sharing)

In [ ]:
# !unzip training

In [ ]:
# !unzip validation

In [22]:
##training

training_path = '/content/train'
validation_path = '/content/valid'

lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir(training_path):
    if not j.startswith('.'):
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'0': 1, '1': 3, '2': 2, '3': 0}

In [23]:
##training

x_train = []
y_train = []
datacount = 0
for i in range(0, 4):
    for j in os.listdir(training_path + '/' + str(i) + '/'):
        if not j.startswith('.'):
            count = 0
            img = Image.open(training_path + '/' + str(i) + '/' + j).convert('RGB')
            img = img.resize((70, 70))
            arr = np.array(img)
            x_train.append(arr) 
            count = count + 1
            y_values_t = np.full((count, 1), lookup[str(i)])
            y_train.append(y_values_t)
            datacount = datacount + count
x_train = np.array(x_train, dtype='float32')
y_train = np.array(y_train)
y_train = y_train.reshape(datacount)

In [26]:
##validation

x_val = []
y_val = []
datacount = 0
for i in range(0, 4):
    for j in os.listdir(validation_path + '/' + str(i) + '/'):
        if not j.startswith('.'):
            count = 0
            img = Image.open(validation_path + '/'  + str(i) + '/' + j).convert('RGB')
            img = img.resize((70, 70))
            arr = np.array(img)
            x_val.append(arr) 
            count = count + 1
            y_values_v = np.full((count, 1), lookup[str(i)])
            y_val.append(y_values_v)
            datacount = datacount + count
x_val = np.array(x_val, dtype='float32')
y_val = np.array(y_val)
y_val = y_val.reshape(datacount)

In [30]:
##new activation func from EfficientNet

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

In [31]:
##removing fully-connected layers to creat model distance

model = enet.EfficientNetB0(include_top=False, input_shape=(70, 70, 3),
                            pooling='avg', weights='imagenet')
x = model.output
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('swish_act')(x)
x = Dropout(0.5)(x)

x = Dense(256)(x)
x = Activation('swish_act')(x)

x = Dense(128)(x)
x = Activation('swish_act')(x)

x = Dense(64)(x)
x = Activation('swish_act')(x)

x = Dense(32)(x)
x = Activation('swish_act')(x)

x = Dense(16)(x)
x = Activation('swish_act')(x)

x = Dense(8)(x)
x = Activation('swish_act')(x)

predictions = Dense(4, activation='softmax')(x)
model_final = Model(inputs = model.input, outputs = predictions)

model_final.load_weights('Final_Model_Weights.h5')

model_final.layers.pop()
model_distance = Model(model_final.input, model_final.layers[-17].output)

In [49]:
##using model.predict_generator instead model_predict because of keras problem

valid_datagen = ImageDataGenerator()

valid_generator = valid_datagen.flow(x_val, y_val, shuffle=False)
encoded_samples = model_distance.predict_generator(valid_generator)

In [70]:
##creating ref vectors with each class samples

vec_1 = encoded_samples[0:400]
vec_3 = encoded_samples[400:800]
vec_2 = encoded_samples[800:1200]
vec_0 = encoded_samples[1200:1600]

In [77]:
##averaging

zero  = np.sum(vec_0, axis=0)/400
one   = np.sum(vec_1, axis=0)/400
two   = np.sum(vec_2, axis=0)/400
three = np.sum(vec_3, axis=0)/400

In [79]:
#saving ref vectors

np.savetxt('fist.csv', zero, delimiter=',')
np.savetxt('left.csv', one, delimiter=',')
np.savetxt('right.csv', two, delimiter=',')
np.savetxt('palm.csv', three, delimiter=',')